In [90]:
import pandas as pd
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [70]:
data = pd.read_csv('transactions1.txt', header = None, names=['date', 'time', 'id_product', 'price', 'count', 'sale', 'id_seller'])

In [71]:
# new column if transaction is canceled
def transaction_canceled(row):
    if row["count"] < 0:
        return 1
    else:
        return 0

In [72]:
data = data.assign(tran_canceled=data.apply(transaction_canceled, axis=1))

In [73]:
data.groupby('tran_canceled').count().sort_values('id_seller', ascending = False)

,date,time,id_product,price,count,sale,id_seller
tran_canceled,,,,,,,
0,1387,1387,1387,1387,1387,1387,1387
1,32,32,32,32,32,32,32


In [74]:
transByDay = data.groupby(['date']).size().reset_index(name='counts')
transByDayCanceled = data.groupby('date')['tran_canceled'].apply(lambda x: (x==1).sum()).reset_index(name='canceled_counts')

In [75]:
dataAgg = pd.merge(transByDay, transByDayCanceled, how = 'inner', on=['date'])

In [76]:
dataAgg['canceled_'] = round(dataAgg['canceled_counts']/dataAgg['counts'],2)

In [77]:
dataset = pd.merge(data, dataAgg, how = 'inner', on=['date'])

In [78]:
dataset['transaction_num'] = dataset.groupby('date').cumcount()

In [79]:
dataset_first_trans = dataset[dataset['transaction_num']==0]

In [80]:
dataset_first_trans.head()

,date,time,id_product,price,count,sale,id_seller,tran_canceled,counts,canceled_counts,canceled_,transaction_num
0,01/07/2010,09:40:15,ITEM1,50,3,150,SALES_PERSON1,0,22,1,0.05,0
22,02/07/2010,09:21:07,ITEM0,30,1,30,SALES_PERSON0,0,26,3,0.12,0
48,03/07/2010,09:52:40,ITEM1,50,2,100,SALES_PERSON1,0,26,0,0.00,0
74,04/07/2010,09:29:57,ITEM0,30,2,60,SALES_PERSON0,0,27,1,0.04,0
101,05/07/2010,10:22:10,ITEM1,50,3,150,SALES_PERSON1,0,20,1,0.05,0


In [96]:
dataset_first_trans['time_int'] = pd.to_numeric(dataset_first_trans.time.str.replace(':',''))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [132]:
dataset_first_trans.describe()

,price,count,sale,tran_canceled,counts,canceled_counts,canceled_,transaction_num,time_int
count,62.000000,62.000000,62.000000,62.0,62.000000,62.000000,62.000000,62.0,62.000000
mean,40.322581,2.516129,100.645161,0.0,22.887097,0.516129,0.023387,0.0,95332.193548
std,10.076387,0.987495,47.421064,0.0,2.211037,0.882283,0.040039,0.0,3426.252433
min,30.000000,1.000000,30.000000,0.0,18.000000,0.000000,0.000000,0.0,90910.000000
25%,30.000000,2.000000,60.000000,0.0,22.000000,0.000000,0.000000,0.0,93030.250000
50%,50.000000,3.000000,90.000000,0.0,22.000000,0.000000,0.000000,0.0,94033.500000
75%,50.000000,3.000000,120.000000,0.0,24.750000,1.000000,0.040000,0.0,95839.250000
max,50.000000,4.000000,200.000000,0.0,27.000000,3.000000,0.140000,0.0,102307.000000


In [82]:
rp.summary_cont(dataset_first_trans['canceled_'].groupby(dataset_first_trans['id_seller']))

,N,Mean,SD,SE,95% Conf.,Interval
id_seller,,,,,,
SALES_PERSON0,31,0.040323,0.048750,0.008756,0.022878,0.057768
SALES_PERSON1,31,0.006452,0.017039,0.003060,0.000354,0.012549


In [127]:
#computing ANOVA table
cw_lm=ols('canceled_ ~ time_int + C(id_seller)', data=dataset_first_trans).fit() #Specify C for Categorical
print(sm.stats.anova_lm(cw_lm, typ=2))

                sum_sq    df         F    PR(>F)
C(id_seller)  0.013246   1.0  9.796640  0.002718
time_int      0.000233   1.0  0.172524  0.679385
Residual      0.079773  59.0       NaN       NaN


In [131]:
# ID_SELLRT - conclusion is that p < 0.05 means on an average the id_seller with cancelled % transactions ==> matters
#TIME - conclusion is that p > 0.05 means on an average the time with cancelled % transactions ==> does not matter